# Загрузка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import sklearn
from sklearn import metrics

In [2]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [5]:
data_ford_price = pd.read_excel('data_ford_price.xlsx') 

In [6]:
data = data_ford_price[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

C:\Users\Олег\AppData\Local\Temp\ipykernel_5288\3156779575.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace = True)


# Отбор признаков

## 1. Метод RFE

In [7]:
from sklearn.feature_selection import RFE

In [8]:
# Выделяем целевой признак
y = data['price']
x = data.drop(columns='price')

In [9]:
# Делим выборки на тренировочные и тестовые
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [11]:
# Выделим три наиболее значимых признака методом RFE
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

In [12]:
# Составим список отобранных признаков:
columns_RFE = ['year', 'cylinders', 'lat']

# Перепишем тренирвочные  и тестовые выборки:
X_train_RFE = X_train[columns_RFE]
X_test_RFE = X_test[columns_RFE]

In [13]:
# Строим модель линейной регрессии:
model = LinearRegression()
# Обучаем модель:
model.fit(X_train_RFE, y_train)
# Делаем предсказание
y_predicted_RFE = model.predict(X_test_RFE)

# Выводим метрики
print(f'MAE = {round(sklearn.metrics.mean_absolute_error(y_test, y_predicted_RFE), 2)}')
print(f'R^2 = {round(sklearn.metrics.r2_score(y_test, y_predicted_RFE), 2)}')

MAE = 5096.57
R^2 = 0.58


**Примечание**. В качестве метрик мы выбрали MAE и R^2, поскольку их легко интерпретировать. Поскольку мы предсказываем цену, то метрика MAE Покажет нам, на сколько тыс. дол. ошибается наша модель. Метрика R^2 покажет, какую долю информации уловила наша модель.

## 2. SelectKBest

In [14]:
from sklearn.feature_selection import SelectKBest, f_regression

In [15]:
# Выделим три наиболее значимых признака методом SelectKBest
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

In [16]:
# Составим список отобранных признаков:
columns_SB = ['year', 'cylinders', 'odometer']

# Перепишем тренирвочные  и тестовые выборки:
X_train_SB = X_train[columns_SB]
X_test_SB = X_test[columns_SB]

In [17]:
# Обучаем модель:
model.fit(X_train_SB, y_train)
# Делаем предсказание
y_predicted_SB = model.predict(X_test_SB)

# Выводим метрики
print(f'MAE = {round(sklearn.metrics.mean_absolute_error(y_test, y_predicted_SB), 2)}')
print(f'R^2 = {round(sklearn.metrics.r2_score(y_test, y_predicted_SB), 2)}')

MAE = 4708.95
R^2 = 0.6


Как видно из метрик, отбор признаков методом SelectKBest показал лучший результат на тестовой выборке.